# Solving Viscous Wave Equation

This notebook is used to solve the particular viscous wave equation for the setup of a muon passing through liquid Xenon. The goal is to find the pressure distribution in a container and its decay as a function of distance and time.

## Wave equation

We can derive the viscous wave equation through the linearised Navier-Stokes Equations. The full derivation is in the notes, but the point is that we have added a single term that takes into account the viscous effects. The equation turns out to be as follows:

$$\Delta \left(p(\vec{x},t) - \frac{1}{\omega_0}\frac{\partial}{\partial t}p(\vec{x},t)\right) = \frac{1}{c^2} \frac{\partial}{\partial}p(\vec{x},t)$$

where $c$ is the speed of the wave in the medium, $\omega_0$ is the coefficient responsible for damping, and $p(\vec{x},t)$ is pressure as a function of space and time.

We now introduce the linear differential operator $\mathcal{L}$.

$$\mathcal{L} := \Delta \left(p(\vec{x},t) - \frac{1}{\omega_0}\frac{\partial}{\partial t}p(\vec{x},t)\right) - \frac{1}{c^2} \frac{\partial}{\partial}p(\vec{x},t)$$

## Fundamental Solution

Now we search for the fundamental solution $F(\vec{x},t)$ that satisfies the follwing.

$$\mathcal{L}F(\vec{x},t) = \delta (\vec{x})\delta(t)$$

We can solve it using fourier methods and then we obtain the fundamental solution in terms of this integral over fourier momentum. Now we just need to evaluate this.

$$F(\vec{x},t) = \Theta(t) \int_0^\infty 8\pi^2 c^2 k^2 e^{-\frac{c^2 k^2}{2 \omega_0}t} \frac{e^{i k r} - e^{-ikr}}{2ikr} \frac{e^{ikct\sqrt{1-\frac{k^2c^2}{4\omega_0}}} - e^{-ikct\sqrt{1-\frac{k^2c^2}{4\omega_0}}}}{2ikct\sqrt{1-\frac{k^2c^2}{4\omega_0}}}dk$$

Where $\Theta(t)$ is the Heaviside function and $r:=\left| \vec{x}\right|$

## Numerical Integration

To numerically evaluate this expression we first need to study it. Therefore let's define the function f as the inner part of the integral like so.

$$F(\vec{x},t) = \Theta(t)\int_0^\infty f(k;\vec{x},t)dk$$

Now we realise that the function f is a piecewise function plotted below.

In [4]:
#import relevant libraries
import numpy as np
import scipy.constants as const
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from tqdm import tqdm

# if this doesn't work run: "python3 -m pip install ipympl" 
# or just comment it out and learn to live without sliders on the graphs
%matplotlib widget       

# Define the relevant constants
c   = 1    # Speed of wave
w_0 = 1    # Damping frequency

# Define integrable function f
def f(k,r,t,w_0=w_0,c=c):
    if t < 0:
        return 0
    
    if k <= 0:
        return 0;
    
    if k < 2*w_0/c:
        return 8 * np.pi**2 * c**2 * k**2 * np.exp(-c**2 * k**2 * t / (2 * w_0)) *\
                np.sinc(k*c*t/np.pi *(1 - k**2 * c**2 / (4 * w_0**2))**0.5) *\
                np.sin(r*k)/(r*k)
    
    elif k > 2*w_0/c:
        return 8 * np.pi**2 * c**2 * k**2 *\
                (np.exp(k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 - c**2 * k**2 * t / (2 * w_0)) -\
                np.exp(-k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 - c**2 * k**2 * t / (2 * w_0)))/\
                (2*k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 ) * np.sin(r*k)/(r*k)
    else: return 0
    
# Define the envelope function of f
def e(k,r,t):
    if t < 0:
        return 0
    
    if k <= 0:
        return 0;
    
    if k < 2*w_0/c:
        return 8 * np.pi**2 * c**2 * np.exp(-c**2 * k**2 * t / (2 * w_0))/\
                (r*c*t *(1 - k**2 * c**2 / (4 * w_0**2))**0.5)
    
    elif k > 2*w_0/c:
        return 8 * np.pi**2 * c**2 *\
                (np.exp(k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 - c**2 * k**2 * t / (2 * w_0)) -\
                np.exp(-k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 - c**2 * k**2 * t / (2 * w_0)))/\
                (2*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 )/r
    else: return 0

In [5]:
# Set up the plot
fig = plt.figure(figsize=(9,6))
fig.suptitle("Integrand plot vs k")
ax = fig.add_subplot(111)

k_min = 0
k_max = 100
Npts = 100

ax.set_xlim([k_min, k_max])
ax.set_ylabel(r'$f(k;r,t)$',fontsize = 15)
ax.set_xlabel(r'$k$',fontsize = 15)
ax.grid(True)

# Generate x-y values
k = np.linspace(k_min,k_max,Npts)
def f_vect(K,r,t):
    return np.array([f(k,r,t) for k in K])

# Add the sliders
@widgets.interact(r=(0.001, 100, 0.1), t=(0.001, 10, .01), Npts=(100,10000, 1))
def update(r =1.0, t=1.0, Npts=1000):
    [l.remove() for l in ax.lines]
    k = np.linspace(k_min,k_max,Npts)
    ax.plot(k, f_vect(k,r,t), color='C3')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

interactive(children=(FloatSlider(value=1.0, description='r', min=0.001), FloatSlider(value=1.0, description='…

# Integration

Now that we have the plot we need to somehow integrate it. To do this efficiently we need to find when does the wave decay over a specific amount. As a result we need to find its envelope. To do this we have to study each of the pieces of the piecewise function seperately.

## Envelope

Now based on the analytic expression of the integrand we can conclude that the envelope function $\varepsilon(k;r,t)$ is the following.

$$
\varepsilon(k;r,t) = \begin{cases}
\frac{8\pi^2 c^2 e^{-\frac{c^2 k^2}{2 \omega_0}t}}{rct\sqrt{1-\frac{k^2c^2}{4\omega_0}}} & \text{if } k \leq \frac{2\omega_0}{c}\\ 
8\pi^2 c^2 e^{-\frac{c^2 k^2}{2 \omega_0}t}\frac{\text{sinh}\left(kct\sqrt{\frac{k^2c^2}{4\omega_0}-1}\right)}{rct\sqrt{\frac{k^2c^2}{4\omega_0}-1}} & \text{if } k > \frac{2\omega_0}{c}\\ 
\end{cases}
$$

Let's try to plot one against the other

In [17]:
# Set up the plot
fig2 = plt.figure(figsize=(9,6))
fig2.suptitle("Integrand plot vs k with envelope")
ax2 = fig2.add_subplot(111)

k_min = 0
k_max = 100
Npts = 100

ax2.set_ylim([-10, 40])
ax2.set_xlim([k_min, k_max])
ax2.set_ylabel(r'$f(k;r,t)$',fontsize = 15)
ax2.set_xlabel(r'$k$',fontsize = 15)
ax2.grid(True)

# Generate x-y values
k2 = np.linspace(k_min,k_max,Npts)
def f_vect(K,r,t):
    return np.array([f(k,r,t) for k in K])

def e_vect(K,r,t):
    return np.array([e(k,r,t) for k in K])

# Add the sliders
@widgets.interact(r=(0.001, 100, 0.1), t=(0.001, 10, .01), Npts=(100,10000, 1))
def update2(r = 1.0, t=1.0, Npts=1000):
    [l.remove() for l in ax2.lines]
    k2 = np.linspace(k_min,k_max,Npts)
    ax2.plot(k2, f_vect(k2,r,t), color='C3')
    ax2.plot(k2, e_vect(k2,r,t), color='C0')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

interactive(children=(FloatSlider(value=1.0, description='r', min=0.001), FloatSlider(value=1.0, description='…

## Approximations

Now it is easy to see how to find a limit for our integration. Specifically we can use this envelope equation to calculate the maximum $k$ to integrate to based on a small number $\delta$ given by the user.

To do this we approximate the envelope for the interval where $k>\frac{2\omega_0}{c}$ by the limit of the numerator like so:

$$ \varepsilon(k;\vec{x},t) \approx \frac{2\omega_0}{c} \sqrt{\frac{16 \pi^4 c^2}{\delta^2 \left|\vec{x}\right|^2t^2}+1}$$

Now the other thing we need to do is to find the step based on the frequency. That should not be hard though because the frequency is only due to a one sin at large $k$'s. Specifically, we know that the frequency f is equal to

$$ f = \frac{r}{2\pi} $$

Therefore now that we know that, we want to analyse the period in $N_{period} = N_p$ points. Therefore here is the number of points per segment as a function of the given varibles

$$N_{pts} = k_0 f N_p = \frac{k_0 r}{2\pi} N_p = \frac{\omega_0 \left|\vec{x}\right| N_p}{c \pi} \sqrt{\frac{16 \pi^4 c^2}{\delta^2 \left|\vec{x}\right|^2t^2}+1}$$

By choosing our variables sufficiently well we can finally figure out the number of operations for the calculation of one point from the integral. This is graphed below.

In [8]:
# Set up the plot
fig3 = plt.figure(figsize=(9,6))
fig3.suptitle("Number of points vs radius")
ax3 = fig3.add_subplot(111)

r_min = 1e-5
r_max = 100
Npts = 1000

ax3.set_xlim([r_min, r_max])
ax3.set_ylabel(r'$N_{pts}$',fontsize = 15)
ax3.set_xlabel(r'$r$',fontsize = 15)
ax3.grid(True)

# Generate x-y values
r = np.linspace(r_min,r_max,Npts)
def Npts_vect(R,delta,N_p,t):
    return np.array([w_0 * r * N_p/(c * np.pi) * (16*np.pi**4 * c**2/(delta**2 * r**2 * t**2)* np.exp(-2*w_0*t) + 1)**0.5 for r in R])

# Add the sliders
@widgets.interact(delta=(1e-5, 1, 1e-5), N_p=(1, 1000, 10), t=(1e-5,100, 0.1))
def update3(delta = 1e-3, N_p=100, t=1):
    [l.remove() for l in ax3.lines]
    N = Npts_vect(r,delta,N_p,t)
    ax3.plot(r, N, color='C3')
    ax3.set_ylim(min(N),max(N))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

interactive(children=(FloatSlider(value=0.001, description='delta', max=1.0, min=1e-05, step=1e-05), IntSlider…

## Performing the integration

Now we are actually going to perform the integration for the wave. To do this we are going to use a very simple integration algoritm by using the 1/3 Simpson's rule applied to each interval to hopefully reduce the number of points evaluated by an order of magnitude or two.

To do this the basic idea is to integrate over an entire volume by estimating the integral between adjacent points $x_i$ and $x_{i+1}$ as shown below.

$$\int_{x_i}^{x_{i+1}} f(x) dx \approx \frac{N_{pts}}{6 k_0} \left[f(x_i) + 4 f\left(\frac{x_i + x_{i+1}}{2}\right) + f(x_{i+1})\right]$$

In [9]:
# Performing Simpson's integration over a predefined function

# First do it between two points
def simps_point(f,a,b,Dx=-1,params=()):
    if Dx<0: Dx = abs(a-b)
    
    if params == ():
        return Dx/6 * (f(a) + 4*f((a+b)/2) + f(b))
    else:
        return Dx/6 * (f(a,*params) + 4*f((a+b)/2,*params) + f(b,*params))

# Now do it for Npts points in an interval
def simps(f,a,b,Npts,progress = False,params = ()):
    Dx = abs(a-b)/Npts
    
    integral = 0
    if progress: 
        for i in tqdm(range(0,int(Npts))): 
            integral += simps_point(f,a,a+i*Dx,Dx,params)
        
    else: 
        for i in range(0,int(Npts)): 
            integral += simps_point(f,a,a+i*Dx,Dx,params)
        
    return integral

In [10]:
# Now that we have our integrator we can attempt to calculate the integral

# Function to calculate the number of points given a particular set of constants
def Npts(r,t,delta,Np,w_0,c):
    return w_0 * r * Np/(c * np.pi) * (16*np.pi**4 * c**2/(delta**2 * r**2 * t**2)* np.exp(-2*w_0*t) + 1)**0.5


#function to calculate the maximum point to integrate based on delta
def k_0(r,t,delta,w_0,c):
    return 2*w_0/c * (16*np.pi**4*c**2/(delta**2 * r**2 * t**2)* np.exp(-2*w_0*t) + 1)**0.5


# Now we are ready to calculate the integral for a single set of parameters
def F(r,t,delta,Np,w_0,c,progress = False):
    return simps(f, 1e-4, k_0(r,t,delta,w_0,c), Npts(r,t,delta,Np,w_0,c), params=(r,t,w_0,c), progress = progress)

# print(F(1,1e-2,5e-3,7,w_0,c,progress = True))

## Plotting the Fundamental Solution

Now that we can calculate 1 point we can calculate a range of points! Therefore why don't we show the fundamental solution over a radius over time? 

Let's try to write a function that can calculate this for a range of values of r form r_min to r_max and t from t_min to t_max with respective dr and dt's. And plot at the end!


In [11]:
def F_range(r_min,r_max,Nr,t_min,t_max,Nt,delta,Np,w_0,c,progress = True):
    
    """Returns a Meshgrid for the 3D representation of wave in r and t"""
    
    print("""
    Simulation the fundamental solution of a sound wave produced by a delta source
    ------------------------------------------------------------------------------
    The simulation will generate a (""",Nr,"""x""",Nt,""") grid that contains the solution
    over the mesh.
    Ranges:
        time: """,t_min,""",""",t_max,"""\tdt =""",(t_max-t_min)/Nt,"""
        dist: """,r_min,""",""",r_max,"""\tdr =""",(r_max-r_min)/Nr,"""
        
    Constants:
        c    =""",c  ,"""
        w_0  =""",w_0,"""
        2w/c =""",2*w_0/c,"""
        
    Integration Params:
        δ      =""",delta,""" 
        k0_m   =""",k_0(r_min,t_min,delta,w_0,c),"""
        k0_M   =""",k_0(r_max,t_min,delta,w_0,c),"""
        Np     =""",Np,"""
        Npts_m =""",int(Npts(r_min,t_max,delta,Np,w_0,c)),"""
        Npts_M =""",int(Npts(r_max,t_min,delta,Np,w_0,c)),"""

    Time remaining:""")
    
    R = np.linspace(r_min,r_max,Nr)
    T = np.linspace(t_min,t_max,Nt)
    
    grid = []
    if progress:
        for t in tqdm(T):
            grid.append([])
            for r in R:
                grid[-1].append(F(r,t,delta,Np,w_0,c))
    else: 
        for t in T:
            grid.append([])
            for r in R:
                grid[-1].append(F(r,t,delta,Np,w_0,c))
                
    return np.array(grid)


def F_slice(r_min,r_max,Nr,t,delta,Np,w_0,c,progress = True):
    
    """Returns The radius range for a particular time t"""
    
    print("""
    Simulation the fundamental solution of a sound wave produced by a delta source
    ------------------------------------------------------------------------------
    The simulation will generate a (""",Nr,"""x""",Nt,""") grid that contains the solution
    over the mesh.
    Ranges:
        time: """,t,"""
        dist: """,r_min,""",""",r_max,"""\tdr =""",(r_max-r_min)/Nr,"""
        
    Constants:
        c    =""",c  ,"""
        w_0  =""",w_0,"""
        2w/c =""",2*w_0/c,"""
        
    Integration Params:
        δ      =""",delta,""" 
        k0_m   =""",k_0(r_min,t,delta,w_0,c),"""
        k0_M   =""",k_0(r_max,t,delta,w_0,c),"""
        Np     =""",Np,"""
        Npts_m =""",int(Npts(r_min,t,delta,Np,w_0,c)),"""
        Npts_M =""",int(Npts(r_max,t,delta,Np,w_0,c)),"""

    Time remaining:""")
    
    R = np.linspace(r_min,r_max,Nr)
    T = np.linspace(t_min,t_max,Nt)
    
    grid = []
    if progress:
        for r in tqdm(R):
            grid.append(F(r,t,delta,Np,w_0,c))
    else: 
        for r in R:
            grid.append(F(r,t,delta,Np,w_0,c))
                
    return np.array(grid)

In [12]:
F_range(0.001,0.005,5,0.1,0.2,5,delta=1e-3,Np=100,w_0=696981,c=1210)

100%|██████████| 5/5 [00:00<00:00, 52.83it/s]
    Simulation the fundamental solution of a sound wave produced by a delta source
    ------------------------------------------------------------------------------
    The simulation will generate a ( 5 x 5 ) grid that contains the solution
    over the mesh.
    Ranges:
        time:  0.1 , 0.2 	dt = 0.02 
        dist:  0.001 , 0.005 	dr = 0.0008 
        
    Constants:
        c    = 1210 
        w_0  = 696981 
        2w/c = 1152.0347107438017 
        
    Integration Params:
        δ      = 0.001  
        k0_m   = 1152.0347107438017 
        k0_M   = 1152.0347107438017 
        Np     = 100 
        Npts_m = 18 
        Npts_M = 91 

    Time remaining:



array([[ 2.78423758e+02,  2.48160593e+02, -9.32126189e+02,
         1.20852099e+05,  8.09205465e+02],
       [ 2.78419936e+02,  2.48156901e+02,  2.70549422e+02,
         7.32315391e+03,  2.25886107e+05],
       [ 2.78415266e+02,  2.48152737e+02,  2.42276989e+02,
        -2.02534731e+03,  1.59796952e+02],
       [ 2.78409745e+02,  2.48147817e+02,  2.42016699e+02,
        -2.54184191e+02, -2.00392374e+04],
       [ 2.78403376e+02,  2.48142140e+02,  2.42096136e+02,
         2.37902083e+02,  2.41539555e+02]])

In [13]:
# Now create a 3D plot of the thingy

def get_meshgrid(r_min,r_max,Nr,t_min,t_max,Nt):
    R = np.linspace(r_min,r_max,Nr)
    T = np.linspace(t_min,t_max,Nt)
    
    return np.meshgrid(R,T)

# Parameters
r_min = 0.0001
r_max = 0.005
Nr    = 100
t_min = 0.1
t_max = 0.2
Nt    = 2

rr, tt = get_meshgrid(r_min,r_max,Nr,t_min,t_max,Nt)
zz = F_range(r_min,r_max,Nr,t_min,t_max,Nt,delta=1e-3,Np=5000,w_0=696981,c=1210)

fig4 = plt.figure()
ax4 = fig4.add_subplot(111, projection='3d')

ax4.plot_surface(rr, tt, zz)

  0%|          | 0/2 [00:00<?, ?it/s]
    Simulation the fundamental solution of a sound wave produced by a delta source
    ------------------------------------------------------------------------------
    The simulation will generate a ( 100 x 2 ) grid that contains the solution
    over the mesh.
    Ranges:
        time:  0.1 , 0.2 	dt = 0.05 
        dist:  0.0001 , 0.005 	dr = 4.9e-05 
        
    Constants:
        c    = 1210 
        w_0  = 696981 
        2w/c = 1152.0347107438017 
        
    Integration Params:
        δ      = 0.001  
        k0_m   = 1152.0347107438017 
        k0_M   = 1152.0347107438017 
        Np     = 5000 
        Npts_m = 91 
        Npts_M = 4583 

    Time remaining:
100%|██████████| 2/2 [00:20<00:00, 10.15s/it]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [14]:
# Set up the plot
fig5 = plt.figure(figsize=(9,6))
fig5.suptitle("Integrand plot vs k")
ax5 = fig5.add_subplot(111)

r_min = 3e-4
r_max = 1e-2
Nr = 100
t = 1

ax5.set_xlim([r_min, r_max])
ax5.set_ylabel(r'$P(r,t) [Pa]$',fontsize = 15)
ax5.set_xlabel(r'$r [m]$',fontsize = 15)
ax5.grid(True)

R = np.linspace(r_min,r_max,Nr)
P = F_slice(r_min,r_max,Nr,t,delta=1e-3,Np=5000,w_0=696981,c=1210,progress = True)

ax5.plot(R,P,color='C3')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

  5%|▌         | 5/100 [00:00<00:02, 47.49it/s]
    Simulation the fundamental solution of a sound wave produced by a delta source
    ------------------------------------------------------------------------------
    The simulation will generate a ( 100 x 2 ) grid that contains the solution
    over the mesh.
    Ranges:
        time:  1 
        dist:  0.0003 , 0.01 	dr = 9.7e-05 
        
    Constants:
        c    = 1210 
        w_0  = 696981 
        2w/c = 1152.0347107438017 
        
    Integration Params:
        δ      = 0.001  
        k0_m   = 1152.0347107438017 
        k0_M   = 1152.0347107438017 
        Np     = 5000 
        Npts_m = 275 
        Npts_M = 9167 

    Time remaining:
100%|██████████| 100/100 [00:20<00:00,  4.95it/s]
